In [ ]:
function dfs_a(arr, now, pla, n, k)
    if pla > n
        return now == k
    end
    
    if dfs_a(arr, now + arr[pla], pla+1, n, k)
        return true
    elseif dfs_a(arr, now, pla+1, n, k)
        return true
    end
    return false 
end

function func_a(arr, k)
    n = length(arr)
    return dfs_a(arr, 0, 1, n, k)
end

In [ ]:
function func_b(arr, k)
    n = length(arr)
    for mask in 0:(1<<n)
        now = 0
        for i in 1:n
            if mask & (1<<(i-1)) != 0
                now += arr[i]
            end
        end
        if now == k
            return true
        end
    end
    return false
end

In [ ]:
arr = [1, 3, 5, 7, 9]
k = 11

In [ ]:
func_a(arr, k)

In [ ]:
func_b(arr, k)

In [ ]:
arr = [1, 3, 5, 7, 9]
k = 2

In [ ]:
func_a(arr, k)

In [ ]:
func_b(arr, k)

In [ ]:
@time func_a(arr, k)

In [ ]:
using BenchmarkTools

In [ ]:
benchmark_result = @benchmark func_a(arr, k)

In [ ]:
mean(benchmark_result)

In [ ]:
mean(benchmark_result).time

In [ ]:
times_a = []
times_b = []
for n in 5:15
    arr = rand(1:100, n)
    k = n*100
    a = @benchmark func_a(arr, k)
    b = @benchmark func_b(arr, k)
    push!(times_a, mean(a).time)
    push!(times_b, mean(b).time)
end

In [ ]:
using Plots

In [ ]:
plot(5:15, times_a, legend = false)
plot!(5:15, times_b, legend = false)

In [ ]:
for i in 1:10
    println(Threads.threadid())
end

In [ ]:
Threads.@threads for i in 1:10
    println(Threads.threadid())
end

In [ ]:
lines = readlines("data/3dh5.pdb")

In [ ]:
atom_line = ""
for line in lines
    if startswith(line, "ATOM")
        atom_line = line
    end
end
atom_line

In [ ]:
atom_line[18:20] # res

In [ ]:
atom_line[31:38] # x

In [ ]:
atom_line[39:46] # y

In [ ]:
atom_line[47:54] # z

In [ ]:
parse(Float64, atom_line[31:38])

In [ ]:
residue_map = Dict(
    "ALA" => "A",
    "ARG" => "R",
    "ASN" => "N",
    "ASP" => "D",
    "CYS" => "C",
    "GLN" => "Q",
    "GLU" => "E",
    "GLY" => "G",
    "HIS" => "H",
    "ILE" => "I",
    "LEU" => "L",
    "LYS" => "K",
    "MET" => "M",
    "PHE" => "F",
    "PRO" => "P",
    "SER" => "S",
    "THR" => "T",
    "TRP" => "W",
    "TYR" => "Y",
    "VAL" => "V",
    "ASX" => "B",
    "XAA" => "X",
    "GLX" => "Z"
)

In [ ]:
function readpdb(file_name; chain_select = ' ')
    x = Array{Float64, 1}()
    y = Array{Float64, 1}()
    z = Array{Float64, 1}()
    res = Array{String, 1}()
    res_count = 0
    natom = 0
    for line in readlines(file_name)
        if !startswith(line, "ATOM") continue end        
        if chain_select != ' ' && line[22] != chain_select continue end
        
        push!(x, parse(Float64, line[31:38]))
        push!(y, parse(Float64, line[39:46]))
        push!(z, parse(Float64, line[47:54]))
                    
        if parse(Int64, line[23:26]) != res_count
            res_count = parse(Int64, line[23:26])
            push!(res, residue_map[line[18:20]])
        end
        
        natom += 1
    end
    return (x = x, y = y, z = z, natom = natom, res = res)
end

In [ ]:
pdb_1un5 = readpdb("data/1un5.pdb", chain_select = 'A')
pdb_3dh5 = readpdb("data/3dh5.pdb", chain_select = 'A')
pdb_2h6x = readpdb("data/2h6x.pdb", chain_select = 'A');

In [ ]:
scatter(pdb_1un5.x, pdb_1un5.y, pdb_1un5.z)

In [ ]:
function get_distance_map(pdb)
    distance_map = zeros(pdb.natom, pdb.natom)
    for i in 1:pdb.natom, j in 1:pdb.natom
        distance_map[i, j] = sqrt((pdb.x[i] - pdb.x[j])^2 + (pdb.y[i] - pdb.y[j])^2 + (pdb.z[i] - pdb.z[j])^2)
    end
    return distance_map
end

In [ ]:
heatmap(get_distance_map(pdb_1un5))

In [ ]:
heatmap(get_distance_map(pdb_2h6x))

In [ ]:
function read_score_talbe(file_name)
    lines = readlines(file_name)
    res_map = Dict()
    res = split(lines[1])
    for i in 1:length(res)
        res_map[res[i]] = i
    end

    table = Array{Int64}(undef, length(res), length(res))

    for i in 2:length(lines)
        rows = split(lines[i])
        for j in 2:length(rows)
            table[i-1, j-1] = parse(Int64, rows[j])
        end
    end
    
    return (map = res_map, table = table)
end

st = read_score_talbe("data/blosum62.txt")

In [ ]:
st.map

In [ ]:
st.table

In [ ]:
get_score(st, res_a, res_b) = st.table[st.map[res_a], st.map[res_b]]

In [ ]:
get_score(st, "A", "W")

In [ ]:
function alignment(st, res_a, res_b)
    length_a = length(res_a)
    length_b = length(res_b)
    dp = [-Inf for i in 1:length_a+1, j in 1:length_b+1]
    from = [(-1, -1) for i in 1:length_a+1, j in 1:length_b+1]
    dp[1, 1] = 0
    for i in 1:length_a+1, j in 1:length_b+1       
        if i <= length_a
            score = dp[i, j] + get_score(st, res_a[i], "*")
            if dp[i+1, j] < score
                dp[i+1, j] = score
                from[i+1, j] = (i, j)
            end
        end
        if j <= length_b
            score = dp[i, j] + get_score(st, "*", res_b[j])
            if dp[i, j+1] < score
                dp[i, j+1] = score
                from[i, j+1] = (i, j)
            end
        end
        if i <= length_a && j <= length_b
            score = dp[i, j] + get_score(st, res_a[i], res_b[j])
            if dp[i+1, j+1] < score
                dp[i+1, j+1] = score
                from[i+1, j+1] = (i, j)
            end
        end
    end
    
    str_a = ""
    str_b = ""
    now = (length_a+1, length_b+1)
    while now != (1, 1)
        pre = from[now[1], now[2]]
        if pre[1] == now[1]
            str_a *= ' '
            str_b *= res_b[pre[2]]
        elseif pre[2] == now[2]
            str_a *= res_a[pre[1]]
            str_b *= ' '
        else
            str_a *= res_a[pre[1]]
            str_b *= res_b[pre[2]]
        end
        
        now = pre
    end
    
    str_a = reverse(str_a)
    str_b = reverse(str_b)
    
    return (score = dp[length_a+1, length_b+1], str_a = str_a, str_b = str_b)
end

In [ ]:
alignment(st, ["K", "E", "T", "A", "A"], ["K", "T", "A", "A"])

In [ ]:
alignment(st, pdb_1un5.res, pdb_1un5.res)

In [ ]:
alignment(st, pdb_1un5.res, pdb_3dh5.res)

In [ ]:
alignment(st, pdb_1un5.res, pdb_2h6x.res)

In [ ]:
alignment(st, pdb_3dh5.res, pdb_3dh5.res)

In [ ]:
alignment(st, pdb_3dh5.res, pdb_2h6x.res)

In [ ]:
alignment(st, pdb_2h6x.res, pdb_2h6x.res)